Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
from itertools import product
import numpy as np
import networkx as nx
import heapq
import random
import time
from typing import List, Tuple, Optional, Dict, Any
import pandas as pd
import warnings

# Suppress NetworkX warnings about Bellman-Ford on large graphs
warnings.filterwarnings("ignore", category=UserWarning) 

# --- 1. PROBLEM GENERATOR ---

def create_problem(
    size: int,
    *,
    density: float = 1.0,
    negative_values: bool = False,
    noise_level: float = 0.0,
    seed: int = 42,
) -> Tuple[np.ndarray, np.ndarray]:
    """Returns cost matrix (problem) and city coordinates (city_coords)."""
    rng = np.random.default_rng(seed)
    city_coords = rng.random(size=(size, 2))
    problem = rng.random((size, size))
    
    if negative_values:
        # Scale values to range [-1, 1]
        problem = problem * 2 - 1
        
    problem *= noise_level
    
    for a, b in product(range(size), repeat=2):
        if rng.random() < density:
            # Calculate Euclidean distance
            dx = city_coords[a, 0] - city_coords[b, 0]
            dy = city_coords[a, 1] - city_coords[b, 1]
            dist = np.sqrt(dx * dx + dy * dy)
            problem[a, b] += dist
        else:
            problem[a, b] = np.inf # No edge

    np.fill_diagonal(problem, 0.0)
    # Scale costs to be larger integers (consistent with typical graph problems)
    return (problem * 1_000).round(), city_coords


# --- 2. BASELINE ALGORITHM (Bellman-Ford) ---

def matrix_to_graph(problem: np.ndarray) -> nx.DiGraph:
    """Convert a problem matrix to a directed NetworkX graph."""
    masked = np.ma.masked_array(problem, mask=np.isinf(problem))
    G = nx.from_numpy_array(masked, create_using=nx.DiGraph)
    return G

def bellman_ford_path(problem: np.ndarray, s: int, d: int) -> Tuple[Optional[List[int]], float]:
    """Baseline shortest path using NetworkX Bellman-Ford."""
    G = matrix_to_graph(problem)
    try:
        path = nx.bellman_ford_path(G, s, d, weight="weight")
        cost = nx.path_weight(G, path, weight="weight")
    except nx.NetworkXNoPath:
        path = None
        cost = np.inf
    except nx.NetworkXUnbounded:
        path = None
        cost = -np.inf # Negative cycle detected
    return path, cost


# --- 3. A* HEURISTIC & SEARCH ALGORITHM ---

def matrix_to_adj(problem: np.ndarray) -> List[List[Tuple[int, float]]]:
    """Convert a problem matrix to adjacency lists for heap-based A*."""
    n = problem.shape[0]
    adj = [[] for _ in range(n)]
    for u in range(n):
        for v in range(n):
            w = problem[u, v]
            if np.isfinite(w) and u != v:
                adj[u].append((v, float(w)))
    return adj

def euclidean_distance_heuristic(u: int, d: int, city_coords: np.ndarray) -> float:
    """Admissible heuristic: straight-line distance (h(n))."""
    pos_u = city_coords[u]
    pos_d = city_coords[d]
    return np.linalg.norm(pos_u - pos_d) * 1_000

def astar(problem: np.ndarray, s: int, d: int, city_coords: np.ndarray) -> Tuple[Optional[List[int]], float]:
    """
    A* search implementation using the Euclidean distance heuristic.
    Returns: (path, cost)
    """
    n = problem.shape[0]
    adj = matrix_to_adj(problem)

    def h(u: int) -> float:
        return euclidean_distance_heuristic(u, d, city_coords)

    # g[n] = cost from start to n
    g = [np.inf] * n
    g[s] = 0.0
    parent = [-1] * n
    # Heap stores (f = g + h, node)
    heap = [(h(s), s)] 

    while heap:
        f_u, u = heapq.heappop(heap)

        # Check for outdated entry due to path relaxation
        if f_u > g[u] + h(u) + 1e-6:
            continue
        
        if u == d:
            break

        for v, w in adj[u]:
            tentative_g = g[u] + w
            if tentative_g < g[v]:
                g[v] = tentative_g
                parent[v] = u
                f_v = tentative_g + h(v)
                heapq.heappush(heap, (f_v, v))

    if not np.isfinite(g[d]):
        return None, np.inf

    # Reconstruct path
    path = []
    cur = d
    while cur != -1:
        path.append(cur)
        if cur == s: break 
        cur = parent[cur]
    path.reverse()
    
    return path, g[d]


# --- 4. MAIN EXECUTION LOOP ---

# Parameters to test
sizes = [10,20, 100, 500,1000]
densities = [0.2,0.5, 0.8,1.0]
noise_levels = [0.0,0.1, 0.5,0.8]
neg_flags = [False, True]

# Only ONE sample per combination
num_pairs_per_problem = 1 
rng = np.random.default_rng(0) 

records: List[Dict[str, Any]] = []
scenario_counter = 1

for negative_values in neg_flags:
    for size in sizes:
        for density in densities:
            for noise_level in noise_levels:
                
                print(f"\n--- Scenario {scenario_counter} (Size: {size}, Density: {density}, Neg: {negative_values}) ---")
                
                problem, city_coords = create_problem(
                    size=size,
                    density=density,
                    noise_level=noise_level,
                    negative_values=negative_values,
                    seed=scenario_counter,
                )

                n = problem.shape[0]
                tried_pairs = 0
                s, d = -1, -1 # Initialize start/destination
                bf_cost = np.nan
                bf_path = None

                # Find a single valid (s, d) pair with a finite cost path
                while tried_pairs < num_pairs_per_problem * 10: 
                    tried_pairs += 1
                    s = int(rng.integers(0, n))
                    d = int(rng.integers(0, n))
                    if s == d: continue

                    bf_path_temp, bf_cost_temp = bellman_ford_path(problem, s, d)
                    
                    if np.isfinite(bf_cost_temp):
                        bf_path, bf_cost = bf_path_temp, bf_cost_temp
                        break
                else:
                    print("  Path sampling failed for this configuration (No finite path found).")
                    scenario_counter += 1
                    continue
                
                # --- Execute the comparison on the single valid path ---
                
                # Bellman-Ford Execution (Time)
                bf_time_start = time.perf_counter()
                bellman_ford_path(problem, s, d)
                bf_time = time.perf_counter() - bf_time_start
                bf_path_str = str(bf_path) if bf_path is not None else 'None'

                records.append({
                    "size": size, "density": density, "noise_level": noise_level, 
                    "negative_values": negative_values, "s": s, "d": d,
                    "algo": "BF", "cost": bf_cost, "time": bf_time, "path": bf_path_str,
                })
                print(f"  BF Result | Cost: {bf_cost:.2f} | Time: {bf_time:.4f}s | Path: {bf_path_str}")


                # A* Search Execution
                t0 = time.perf_counter()
                a_path, a_cost = astar(problem, s, d, city_coords)
                t1 = time.perf_counter()
                a_time = t1 - t0
                a_path_str = str(a_path) if a_path is not None else 'None'
                
                records.append({
                    "size": size, "density": density, "noise_level": noise_level, 
                    "negative_values": negative_values, "s": s, "d": d,
                    "algo": "A*", "cost": a_cost, "time": a_time, "path": a_path_str,
                })
                print(f"  A* Result | Cost: {a_cost:.2f} | Time: {a_time:.4f}s | Path: {a_path_str}")

                scenario_counter += 1


# 5. FINAL SUMMARY AND OUTPUT
df = pd.DataFrame(records)

# Filter for successful runs and algorithms
df_success = df[df['algo'].isin(['BF', 'A*'])].copy()

# Pivot table for final summary
summary = df_success.pivot_table(
    index=["size", "density", "noise_level", "negative_values", "s", "d"],
    columns="algo",
    values=["cost", "time", "path"],
    aggfunc='first'
)

# Flatten columns
summary.columns = [f'{col[0]}_{col[1]}' for col in summary.columns]
summary.reset_index(inplace=True)

# Rename columns for clear output
summary.rename(columns={
    'cost_BF': 'BF Cost', 'time_BF': 'BF Time (s)', 'path_BF': 'BF Path',
    'cost_A*': 'A* Cost', 'time_A*': 'A* Time (s)', 'path_A*': 'A* Path',
    's': 'Start', 'd': 'Goal'
}, inplace=True)

# Select columns for the final output (excluding error)
final_cols = [
    'size', 'density', 'noise_level', 'negative_values', 'Start', 'Goal', 
    'BF Cost', 'A* Cost', 'BF Time (s)', 'A* Time (s)', 'BF Path', 'A* Path'
]

summary = summary[final_cols]

# Export to CSV
summary.to_csv("shortest_path_comparison_summary.csv", index=False)

print(f"\n{'='*150}")
print("Final Summary: Bellman-Ford (BF) vs. A* (One Sample Per Case)")
print(f"{'='*150}")

# Print as Markdown
print(summary.to_markdown(index=False, floatfmt='.2f'))
print("\n---")
print("Data saved to shortest_path_comparison_summary.csv.")


--- Scenario 1 (Size: 10, Density: 0.2, Neg: False) ---
  BF Result | Cost: 492.00 | Time: 0.0002s | Path: [5, 6]
  A* Result | Cost: 492.00 | Time: 0.0002s | Path: [5, 6]

--- Scenario 2 (Size: 10, Density: 0.2, Neg: False) ---
  BF Result | Cost: 1519.00 | Time: 0.0002s | Path: [9, 8, 6, 7]
  A* Result | Cost: 1519.00 | Time: 0.0003s | Path: [9, 8, 6, 7]

--- Scenario 3 (Size: 10, Density: 0.2, Neg: False) ---
  BF Result | Cost: 2210.00 | Time: 0.0002s | Path: [2, 6, 1, 8]
  A* Result | Cost: 2210.00 | Time: 0.0002s | Path: [2, 6, 1, 8]

--- Scenario 4 (Size: 10, Density: 0.2, Neg: False) ---
  BF Result | Cost: 1770.00 | Time: 0.0002s | Path: [3, 5, 6, 8]
  A* Result | Cost: 1770.00 | Time: 0.0002s | Path: [3, 5, 6, 8]

--- Scenario 5 (Size: 10, Density: 0.5, Neg: False) ---
  BF Result | Cost: 886.00 | Time: 0.0003s | Path: [5, 8, 0]
  A* Result | Cost: 886.00 | Time: 0.0003s | Path: [5, 8, 0]

--- Scenario 6 (Size: 10, Density: 0.5, Neg: False) ---
  BF Result | Cost: 989.00 | T

  BF Result | Cost: 858.00 | Time: 0.0309s | Path: [89, 45, 79]
  A* Result | Cost: 858.00 | Time: 0.0105s | Path: [89, 45, 79]

--- Scenario 48 (Size: 100, Density: 1.0, Neg: False) ---
  BF Result | Cost: 865.00 | Time: 0.0318s | Path: [70, 35, 23]
  A* Result | Cost: 865.00 | Time: 0.0112s | Path: [70, 35, 23]

--- Scenario 49 (Size: 500, Density: 0.2, Neg: False) ---
  BF Result | Cost: 792.00 | Time: 0.1655s | Path: [383, 180, 26]
  A* Result | Cost: 792.00 | Time: 0.2206s | Path: [383, 180, 26]

--- Scenario 50 (Size: 500, Density: 0.2, Neg: False) ---
  BF Result | Cost: 565.00 | Time: 0.2210s | Path: [285, 352, 202]
  A* Result | Cost: 565.00 | Time: 0.2450s | Path: [285, 352, 202]

--- Scenario 51 (Size: 500, Density: 0.2, Neg: False) ---
  BF Result | Cost: 1136.00 | Time: 0.1814s | Path: [498, 22, 188, 99]
  A* Result | Cost: 1136.00 | Time: 0.2225s | Path: [498, 22, 188, 99]

--- Scenario 52 (Size: 500, Density: 0.2, Neg: False) ---
  BF Result | Cost: 977.00 | Time: 0.1730

  Path sampling failed for this configuration (No finite path found).

--- Scenario 115 (Size: 100, Density: 0.2, Neg: True) ---
  Path sampling failed for this configuration (No finite path found).

--- Scenario 116 (Size: 100, Density: 0.2, Neg: True) ---
  Path sampling failed for this configuration (No finite path found).

--- Scenario 117 (Size: 100, Density: 0.5, Neg: True) ---
  BF Result | Cost: 518.00 | Time: 0.0151s | Path: [65, 37, 97]
  A* Result | Cost: 518.00 | Time: 0.0089s | Path: [65, 37, 97]

--- Scenario 118 (Size: 100, Density: 0.5, Neg: True) ---
  Path sampling failed for this configuration (No finite path found).

--- Scenario 119 (Size: 100, Density: 0.5, Neg: True) ---
  Path sampling failed for this configuration (No finite path found).

--- Scenario 120 (Size: 100, Density: 0.5, Neg: True) ---
  Path sampling failed for this configuration (No finite path found).

--- Scenario 121 (Size: 100, Density: 0.8, Neg: True) ---
  BF Result | Cost: 402.00 | Time: 0.02